In [1]:
from __future__ import print_function
import paddle
import paddle.fluid as fluid
import numpy as np
import sys
import math

CLASS_DIM = 2 #Number of categories for sentiment analysis
EMB_DIM = 64 #Dimensions of the word vector
HID_DIM = 64 #Dimensions of hide layer
STACKED_NUM = 2 #LSTM Layers of the bidirectional stack
BATCH_SIZE = 64 #batch size
EPOCH = 20
LEARNING_RATE=0.0001

In [2]:
#Textconvolution neural network
def convolution_net(data, input_dim, class_dim, emb_dim, hid_dim):
    emb = fluid.layers.embedding(
        input=data, size=[input_dim, emb_dim], is_sparse=True)
    conv_3 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,
        filter_size=3,
        act="tanh",
        pool_type="sqrt")
    conv_4 = fluid.nets.sequence_conv_pool(
        input=emb,
        num_filters=hid_dim,
        filter_size=4,
        act="tanh",
        pool_type="sqrt")
    prediction = fluid.layers.fc(
        input=[conv_3, conv_4], size=class_dim, act="softmax")
    return prediction

In [3]:
#Stack Bidirectional LSTM
def stacked_lstm_net(data, input_dim, class_dim, emb_dim, hid_dim, stacked_num):

    # Calculate word vectorvector
    emb = fluid.layers.embedding(input=data, size=[input_dim, emb_dim], is_sparse=True)

    #First stack
    #Fully connected layer
    fc1 = fluid.layers.fc(input=emb, size=hid_dim)
    #lstm layer
    lstm1, cell1 = fluid.layers.dynamic_lstm(input=fc1, size=hid_dim)

    inputs = [fc1, lstm1]

    #All remaining stack structures
    for i in range(2, stacked_num + 1):
        fc = fluid.layers.fc(input=inputs, size=hid_dim)
        lstm, cell = fluid.layers.dynamic_lstm(input=fc, size=hid_dim, is_reverse=(i % 2) == 0)
        inputs = [fc, lstm]

    #pooling layer
    fc_last = fluid.layers.sequence_pool(input=inputs[0], pool_type='max')
    lstm_last = fluid.layers.sequence_pool(input=inputs[1], pool_type='max')

    #Fully connected layer, softmax prediction
    prediction = fluid.layers.fc(
        input=[fc_last, lstm_last], size=class_dim, act='softmax')
    return prediction

In [4]:
def inference_program(word_dict):
    data = fluid.layers.data(
        name="words", shape=[1], dtype="int64", lod_level=1)

    dict_dim = len(word_dict)
#     net = convolution_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM)
    net = stacked_lstm_net(data, dict_dim, CLASS_DIM, EMB_DIM, HID_DIM, STACKED_NUM)
    return net

In [5]:
def train_program(prediction):
    label = fluid.layers.data(name="label", shape=[1], dtype="int64")
    cost = fluid.layers.cross_entropy(input=prediction, label=label)
    avg_cost = fluid.layers.mean(cost)
    accuracy = fluid.layers.accuracy(input=prediction, label=label)
    return [avg_cost, accuracy] #return average cost and accuracy acc

#Optimization function
def optimizer_func():
#     return fluid.optimizer.Adagrad(learning_rate=LEARNING_RATE)
    return fluid.optimizer.Adam(learning_rate=LEARNING_RATE)

In [6]:
use_cuda = True 
place = fluid.CUDAPlace(1) if use_cuda else fluid.CPUPlace()

In [7]:
print("Loading IMDB word dict....")
word_dict = paddle.dataset.imdb.word_dict()

print ("Reading training data....")
train_reader = paddle.batch(
    paddle.reader.shuffle(
        paddle.dataset.imdb.train(word_dict), buf_size=25000),
    batch_size=BATCH_SIZE)
print("Reading testing data....")
test_reader = paddle.batch(
    paddle.dataset.imdb.test(word_dict), batch_size=BATCH_SIZE)

Loading IMDB word dict....
Reading training data....
Reading testing data....


In [8]:
len(word_dict)

5147

In [9]:
exe = fluid.Executor(place)
prediction = inference_program(word_dict)
[avg_cost, accuracy] = train_program(prediction)#training program
sgd_optimizer = optimizer_func()# training optimization function
sgd_optimizer.minimize(avg_cost)

([inputs {
    parameter: "Beta1Pow"
    arguments: "embedding_0.w_0_beta1_pow_acc_0"
  }
  inputs {
    parameter: "Beta1Tensor"
  }
  inputs {
    parameter: "Beta2Pow"
    arguments: "embedding_0.w_0_beta2_pow_acc_0"
  }
  inputs {
    parameter: "Beta2Tensor"
  }
  inputs {
    parameter: "Grad"
    arguments: "embedding_0.w_0@GRAD"
  }
  inputs {
    parameter: "LearningRate"
    arguments: "learning_rate_0"
  }
  inputs {
    parameter: "Moment1"
    arguments: "embedding_0.w_0_moment1_0"
  }
  inputs {
    parameter: "Moment2"
    arguments: "embedding_0.w_0_moment2_0"
  }
  inputs {
    parameter: "Param"
    arguments: "embedding_0.w_0"
  }
  outputs {
    parameter: "Beta1PowOut"
    arguments: "embedding_0.w_0_beta1_pow_acc_0"
  }
  outputs {
    parameter: "Beta2PowOut"
    arguments: "embedding_0.w_0_beta2_pow_acc_0"
  }
  outputs {
    parameter: "Moment1Out"
    arguments: "embedding_0.w_0_moment1_0"
  }
  outputs {
    parameter: "Moment2Out"
    arguments: "embedding_0

In [10]:
def train_test(program, reader):
    count = 0
    feed_var_list = [
        program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder_test = fluid.DataFeeder(feed_list=feed_var_list, place=place)
    test_exe = fluid.Executor(place)
    accumulated = len([avg_cost, accuracy]) * [0]
    for test_data in reader():
        avg_cost_np = test_exe.run(
            program=program,
            feed=feeder_test.feed(test_data),
            fetch_list=[avg_cost, accuracy])
        accumulated = [
            x[0] + x[1][0] for x in zip(accumulated, avg_cost_np)
        ]
        count += 1
    return [x / count for x in accumulated]

In [11]:
import time

# Specify the directory path to save the parameters
params_dirname = "model/sentiment_lstm.inference.model"
RESULT_FILE = 'result/results_lstm.txt'

feed_order = ['words', 'label']

# Main loop part of the program
def train_loop(main_program):
    # Start the trainer built above
    exe.run(fluid.default_startup_program())

    feed_var_list_loop = [
        main_program.global_block().var(var_name) for var_name in feed_order
    ]
    feeder = fluid.DataFeeder(
        feed_list=feed_var_list_loop, place=place)

    test_program = fluid.default_main_program().clone(for_test=True)

    # Training loop
    start_time = time.time()
    for epoch_id in range(EPOCH):
        for step_id, data in enumerate(train_reader()):
            # Running trainer
            metrics = exe.run(main_program,
                              feed=feeder.feed(data),
                              fetch_list=[avg_cost, accuracy])


            print("Step {0}, Epoch {1} Metrics {2}".format(
                step_id, epoch_id, list(map(np.array,
                                            metrics))))

#             if step_id == 30:
#                 if params_dirname is not None:
#                     fluid.io.save_inference_model(params_dirname, ["words"],
#                                                   prediction, exe)# Save model
#                 return
            
    total_time = time.time() - start_time
    with open(RESULT_FILE, 'a') as f:
        f.write('\n\n\ntraining results: \n')
        f.write('\n total training time: \t {}'.format(total_time))

    start_time = time.time()
    # Testing Results
    avg_cost_test, acc_test = train_test(test_program, test_reader)
    total_time = time.time() - start_time
    s = 'Test Loss {:0.2}, Acc {:0.2}, total time: {}'.format(avg_cost_test, acc_test, total_time)
    print(s)
    with open(RESULT_FILE, 'a') as f:
        f.write('\ntest results: \n')
        f.write(s)

In [12]:
train_loop(fluid.default_main_program())

Step 0, Epoch 0 Metrics [array([0.6776115], dtype=float32), array([0.625], dtype=float32)]
Step 1, Epoch 0 Metrics [array([0.7023661], dtype=float32), array([0.453125], dtype=float32)]
Step 2, Epoch 0 Metrics [array([0.6895298], dtype=float32), array([0.546875], dtype=float32)]
Step 3, Epoch 0 Metrics [array([0.66559213], dtype=float32), array([0.703125], dtype=float32)]
Step 4, Epoch 0 Metrics [array([0.68889034], dtype=float32), array([0.546875], dtype=float32)]
Step 5, Epoch 0 Metrics [array([0.6861359], dtype=float32), array([0.5625], dtype=float32)]
Step 6, Epoch 0 Metrics [array([0.7063793], dtype=float32), array([0.453125], dtype=float32)]
Step 7, Epoch 0 Metrics [array([0.6971178], dtype=float32), array([0.484375], dtype=float32)]
Step 8, Epoch 0 Metrics [array([0.6897936], dtype=float32), array([0.546875], dtype=float32)]
Step 9, Epoch 0 Metrics [array([0.69576895], dtype=float32), array([0.5], dtype=float32)]
Step 10, Epoch 0 Metrics [array([0.70352364], dtype=float32), array

Step 91, Epoch 0 Metrics [array([0.68731904], dtype=float32), array([0.640625], dtype=float32)]
Step 92, Epoch 0 Metrics [array([0.685233], dtype=float32), array([0.734375], dtype=float32)]
Step 93, Epoch 0 Metrics [array([0.6852757], dtype=float32), array([0.765625], dtype=float32)]
Step 94, Epoch 0 Metrics [array([0.6827712], dtype=float32), array([0.734375], dtype=float32)]
Step 95, Epoch 0 Metrics [array([0.68556374], dtype=float32), array([0.71875], dtype=float32)]
Step 96, Epoch 0 Metrics [array([0.6836245], dtype=float32), array([0.734375], dtype=float32)]
Step 97, Epoch 0 Metrics [array([0.68217456], dtype=float32), array([0.734375], dtype=float32)]
Step 98, Epoch 0 Metrics [array([0.68414986], dtype=float32), array([0.71875], dtype=float32)]
Step 99, Epoch 0 Metrics [array([0.68329287], dtype=float32), array([0.734375], dtype=float32)]
Step 100, Epoch 0 Metrics [array([0.6846498], dtype=float32), array([0.703125], dtype=float32)]
Step 101, Epoch 0 Metrics [array([0.6840887], d

Step 179, Epoch 0 Metrics [array([0.6687224], dtype=float32), array([0.828125], dtype=float32)]
Step 180, Epoch 0 Metrics [array([0.6670217], dtype=float32), array([0.765625], dtype=float32)]
Step 181, Epoch 0 Metrics [array([0.6718763], dtype=float32), array([0.78125], dtype=float32)]
Step 182, Epoch 0 Metrics [array([0.67433757], dtype=float32), array([0.671875], dtype=float32)]
Step 183, Epoch 0 Metrics [array([0.6647557], dtype=float32), array([0.8125], dtype=float32)]
Step 184, Epoch 0 Metrics [array([0.6710664], dtype=float32), array([0.75], dtype=float32)]
Step 185, Epoch 0 Metrics [array([0.6702579], dtype=float32), array([0.828125], dtype=float32)]
Step 186, Epoch 0 Metrics [array([0.6719583], dtype=float32), array([0.734375], dtype=float32)]
Step 187, Epoch 0 Metrics [array([0.667809], dtype=float32), array([0.796875], dtype=float32)]
Step 188, Epoch 0 Metrics [array([0.6682062], dtype=float32), array([0.75], dtype=float32)]
Step 189, Epoch 0 Metrics [array([0.6645286], dtype

Step 268, Epoch 0 Metrics [array([0.6518526], dtype=float32), array([0.765625], dtype=float32)]
Step 269, Epoch 0 Metrics [array([0.63903165], dtype=float32), array([0.828125], dtype=float32)]
Step 270, Epoch 0 Metrics [array([0.6349388], dtype=float32), array([0.8125], dtype=float32)]
Step 271, Epoch 0 Metrics [array([0.6391783], dtype=float32), array([0.875], dtype=float32)]
Step 272, Epoch 0 Metrics [array([0.6386317], dtype=float32), array([0.828125], dtype=float32)]
Step 273, Epoch 0 Metrics [array([0.6365462], dtype=float32), array([0.8125], dtype=float32)]
Step 274, Epoch 0 Metrics [array([0.6426037], dtype=float32), array([0.828125], dtype=float32)]
Step 275, Epoch 0 Metrics [array([0.63545233], dtype=float32), array([0.796875], dtype=float32)]
Step 276, Epoch 0 Metrics [array([0.63110137], dtype=float32), array([0.875], dtype=float32)]
Step 277, Epoch 0 Metrics [array([0.6339796], dtype=float32), array([0.765625], dtype=float32)]
Step 278, Epoch 0 Metrics [array([0.6529827], d

Step 357, Epoch 0 Metrics [array([0.5718061], dtype=float32), array([0.84375], dtype=float32)]
Step 358, Epoch 0 Metrics [array([0.5627636], dtype=float32), array([0.859375], dtype=float32)]
Step 359, Epoch 0 Metrics [array([0.56391525], dtype=float32), array([0.828125], dtype=float32)]
Step 360, Epoch 0 Metrics [array([0.58260274], dtype=float32), array([0.78125], dtype=float32)]
Step 361, Epoch 0 Metrics [array([0.56617284], dtype=float32), array([0.859375], dtype=float32)]
Step 362, Epoch 0 Metrics [array([0.5826332], dtype=float32), array([0.8125], dtype=float32)]
Step 363, Epoch 0 Metrics [array([0.5986695], dtype=float32), array([0.75], dtype=float32)]
Step 364, Epoch 0 Metrics [array([0.5671277], dtype=float32), array([0.796875], dtype=float32)]
Step 365, Epoch 0 Metrics [array([0.5985758], dtype=float32), array([0.75], dtype=float32)]
Step 366, Epoch 0 Metrics [array([0.56500304], dtype=float32), array([0.90625], dtype=float32)]
Step 367, Epoch 0 Metrics [array([0.57603514], dt

...

Step 378, Epoch 18 Metrics [array([0.17738873], dtype=float32), array([0.90625], dtype=float32)]
Step 379, Epoch 18 Metrics [array([0.13223818], dtype=float32), array([0.9375], dtype=float32)]
Step 380, Epoch 18 Metrics [array([0.08605541], dtype=float32), array([0.953125], dtype=float32)]
Step 381, Epoch 18 Metrics [array([0.1124342], dtype=float32), array([0.953125], dtype=float32)]
Step 382, Epoch 18 Metrics [array([0.17526424], dtype=float32), array([0.921875], dtype=float32)]
Step 383, Epoch 18 Metrics [array([0.1235249], dtype=float32), array([0.953125], dtype=float32)]
Step 384, Epoch 18 Metrics [array([0.14790452], dtype=float32), array([0.96875], dtype=float32)]
Step 385, Epoch 18 Metrics [array([0.12333968], dtype=float32), array([0.953125], dtype=float32)]
Step 386, Epoch 18 Metrics [array([0.0340932], dtype=float32), array([1.], dtype=float32)]
Step 387, Epoch 18 Metrics [array([0.09213862], dtype=float32), array([0.96875], dtype=float32)]
Step 388, Epoch 18 Metrics [array(

Step 75, Epoch 19 Metrics [array([0.1323276], dtype=float32), array([0.96875], dtype=float32)]
Step 76, Epoch 19 Metrics [array([0.05532199], dtype=float32), array([0.984375], dtype=float32)]
Step 77, Epoch 19 Metrics [array([0.06565817], dtype=float32), array([0.984375], dtype=float32)]
Step 78, Epoch 19 Metrics [array([0.10706797], dtype=float32), array([0.984375], dtype=float32)]
Step 79, Epoch 19 Metrics [array([0.10358988], dtype=float32), array([0.953125], dtype=float32)]
Step 80, Epoch 19 Metrics [array([0.31519043], dtype=float32), array([0.921875], dtype=float32)]
Step 81, Epoch 19 Metrics [array([0.12300909], dtype=float32), array([0.96875], dtype=float32)]
Step 82, Epoch 19 Metrics [array([0.15000802], dtype=float32), array([0.921875], dtype=float32)]
Step 83, Epoch 19 Metrics [array([0.07939691], dtype=float32), array([0.96875], dtype=float32)]
Step 84, Epoch 19 Metrics [array([0.19585192], dtype=float32), array([0.9375], dtype=float32)]
Step 85, Epoch 19 Metrics [array([0.

Step 163, Epoch 19 Metrics [array([0.05892375], dtype=float32), array([0.984375], dtype=float32)]
Step 164, Epoch 19 Metrics [array([0.11098775], dtype=float32), array([0.96875], dtype=float32)]
Step 165, Epoch 19 Metrics [array([0.13607289], dtype=float32), array([0.96875], dtype=float32)]
Step 166, Epoch 19 Metrics [array([0.04232123], dtype=float32), array([0.984375], dtype=float32)]
Step 167, Epoch 19 Metrics [array([0.15222755], dtype=float32), array([0.953125], dtype=float32)]
Step 168, Epoch 19 Metrics [array([0.04960728], dtype=float32), array([0.984375], dtype=float32)]
Step 169, Epoch 19 Metrics [array([0.13026974], dtype=float32), array([0.953125], dtype=float32)]
Step 170, Epoch 19 Metrics [array([0.1544788], dtype=float32), array([0.9375], dtype=float32)]
Step 171, Epoch 19 Metrics [array([0.16503334], dtype=float32), array([0.96875], dtype=float32)]
Step 172, Epoch 19 Metrics [array([0.06036894], dtype=float32), array([0.984375], dtype=float32)]
Step 173, Epoch 19 Metrics

Step 249, Epoch 19 Metrics [array([0.34881353], dtype=float32), array([0.84375], dtype=float32)]
Step 250, Epoch 19 Metrics [array([0.40787995], dtype=float32), array([0.8125], dtype=float32)]
Step 251, Epoch 19 Metrics [array([0.41809633], dtype=float32), array([0.828125], dtype=float32)]
Step 252, Epoch 19 Metrics [array([0.25743872], dtype=float32), array([0.890625], dtype=float32)]
Step 253, Epoch 19 Metrics [array([0.30810302], dtype=float32), array([0.859375], dtype=float32)]
Step 254, Epoch 19 Metrics [array([0.22951166], dtype=float32), array([0.90625], dtype=float32)]
Step 255, Epoch 19 Metrics [array([0.19481039], dtype=float32), array([0.9375], dtype=float32)]
Step 256, Epoch 19 Metrics [array([0.2265273], dtype=float32), array([0.890625], dtype=float32)]
Step 257, Epoch 19 Metrics [array([0.30427378], dtype=float32), array([0.90625], dtype=float32)]
Step 258, Epoch 19 Metrics [array([0.16911224], dtype=float32), array([0.921875], dtype=float32)]
Step 259, Epoch 19 Metrics [

Step 334, Epoch 19 Metrics [array([0.15337446], dtype=float32), array([0.953125], dtype=float32)]
Step 335, Epoch 19 Metrics [array([0.16268167], dtype=float32), array([0.9375], dtype=float32)]
Step 336, Epoch 19 Metrics [array([0.22260526], dtype=float32), array([0.9375], dtype=float32)]
Step 337, Epoch 19 Metrics [array([0.07669708], dtype=float32), array([0.984375], dtype=float32)]
Step 338, Epoch 19 Metrics [array([0.26224998], dtype=float32), array([0.875], dtype=float32)]
Step 339, Epoch 19 Metrics [array([0.09021205], dtype=float32), array([0.984375], dtype=float32)]
Step 340, Epoch 19 Metrics [array([0.10768341], dtype=float32), array([0.96875], dtype=float32)]
Step 341, Epoch 19 Metrics [array([0.17506365], dtype=float32), array([0.9375], dtype=float32)]
Step 342, Epoch 19 Metrics [array([0.18317285], dtype=float32), array([0.9375], dtype=float32)]
Step 343, Epoch 19 Metrics [array([0.17076129], dtype=float32), array([0.9375], dtype=float32)]
Step 344, Epoch 19 Metrics [array(

In [13]:
train_loop(fluid.default_main_program())

Step 0, Epoch 0 Metrics [array([0.69307274], dtype=float32), array([0.46875], dtype=float32)]
Step 1, Epoch 0 Metrics [array([0.69207746], dtype=float32), array([0.5], dtype=float32)]
Step 2, Epoch 0 Metrics [array([0.69382566], dtype=float32), array([0.53125], dtype=float32)]
Step 3, Epoch 0 Metrics [array([0.69351476], dtype=float32), array([0.5], dtype=float32)]
Step 4, Epoch 0 Metrics [array([0.69136035], dtype=float32), array([0.515625], dtype=float32)]
Step 5, Epoch 0 Metrics [array([0.69411224], dtype=float32), array([0.46875], dtype=float32)]
Step 6, Epoch 0 Metrics [array([0.69059944], dtype=float32), array([0.515625], dtype=float32)]
Step 7, Epoch 0 Metrics [array([0.69101], dtype=float32), array([0.578125], dtype=float32)]
Step 8, Epoch 0 Metrics [array([0.6953431], dtype=float32), array([0.453125], dtype=float32)]
Step 9, Epoch 0 Metrics [array([0.6937244], dtype=float32), array([0.515625], dtype=float32)]
Step 10, Epoch 0 Metrics [array([0.6891063], dtype=float32), array([

Step 88, Epoch 0 Metrics [array([0.68669105], dtype=float32), array([0.671875], dtype=float32)]
Step 89, Epoch 0 Metrics [array([0.6773421], dtype=float32), array([0.828125], dtype=float32)]
Step 90, Epoch 0 Metrics [array([0.6829965], dtype=float32), array([0.6875], dtype=float32)]
Step 91, Epoch 0 Metrics [array([0.6819438], dtype=float32), array([0.75], dtype=float32)]
Step 92, Epoch 0 Metrics [array([0.67685527], dtype=float32), array([0.796875], dtype=float32)]
Step 93, Epoch 0 Metrics [array([0.68301904], dtype=float32), array([0.71875], dtype=float32)]
Step 94, Epoch 0 Metrics [array([0.6819483], dtype=float32), array([0.703125], dtype=float32)]
Step 95, Epoch 0 Metrics [array([0.67932475], dtype=float32), array([0.71875], dtype=float32)]
Step 96, Epoch 0 Metrics [array([0.67687833], dtype=float32), array([0.8125], dtype=float32)]
Step 97, Epoch 0 Metrics [array([0.6786413], dtype=float32), array([0.78125], dtype=float32)]
Step 98, Epoch 0 Metrics [array([0.67823863], dtype=floa

Step 178, Epoch 0 Metrics [array([0.6560898], dtype=float32), array([0.796875], dtype=float32)]
Step 179, Epoch 0 Metrics [array([0.6421561], dtype=float32), array([0.828125], dtype=float32)]
Step 180, Epoch 0 Metrics [array([0.6515993], dtype=float32), array([0.8125], dtype=float32)]
Step 181, Epoch 0 Metrics [array([0.6580831], dtype=float32), array([0.765625], dtype=float32)]
Step 182, Epoch 0 Metrics [array([0.6570834], dtype=float32), array([0.796875], dtype=float32)]
Step 183, Epoch 0 Metrics [array([0.6530582], dtype=float32), array([0.8125], dtype=float32)]
Step 184, Epoch 0 Metrics [array([0.6636629], dtype=float32), array([0.734375], dtype=float32)]
Step 185, Epoch 0 Metrics [array([0.65271914], dtype=float32), array([0.75], dtype=float32)]
Step 186, Epoch 0 Metrics [array([0.65758514], dtype=float32), array([0.734375], dtype=float32)]
Step 187, Epoch 0 Metrics [array([0.6562308], dtype=float32), array([0.78125], dtype=float32)]
Step 188, Epoch 0 Metrics [array([0.64631927], 

Step 266, Epoch 0 Metrics [array([0.63289595], dtype=float32), array([0.671875], dtype=float32)]
Step 267, Epoch 0 Metrics [array([0.61612034], dtype=float32), array([0.78125], dtype=float32)]
Step 268, Epoch 0 Metrics [array([0.6373713], dtype=float32), array([0.75], dtype=float32)]
Step 269, Epoch 0 Metrics [array([0.6117939], dtype=float32), array([0.828125], dtype=float32)]
Step 270, Epoch 0 Metrics [array([0.6232681], dtype=float32), array([0.765625], dtype=float32)]
Step 271, Epoch 0 Metrics [array([0.61797017], dtype=float32), array([0.6875], dtype=float32)]
Step 272, Epoch 0 Metrics [array([0.6101116], dtype=float32), array([0.796875], dtype=float32)]
Step 273, Epoch 0 Metrics [array([0.6190524], dtype=float32), array([0.765625], dtype=float32)]
Step 274, Epoch 0 Metrics [array([0.5801487], dtype=float32), array([0.84375], dtype=float32)]
Step 275, Epoch 0 Metrics [array([0.6210151], dtype=float32), array([0.78125], dtype=float32)]
Step 276, Epoch 0 Metrics [array([0.6322615], 

Step 352, Epoch 0 Metrics [array([0.5630509], dtype=float32), array([0.8125], dtype=float32)]
Step 353, Epoch 0 Metrics [array([0.5190736], dtype=float32), array([0.84375], dtype=float32)]
Step 354, Epoch 0 Metrics [array([0.5767566], dtype=float32), array([0.734375], dtype=float32)]
Step 355, Epoch 0 Metrics [array([0.57875764], dtype=float32), array([0.734375], dtype=float32)]
Step 356, Epoch 0 Metrics [array([0.58213806], dtype=float32), array([0.71875], dtype=float32)]
Step 357, Epoch 0 Metrics [array([0.5858102], dtype=float32), array([0.765625], dtype=float32)]
Step 358, Epoch 0 Metrics [array([0.56311077], dtype=float32), array([0.78125], dtype=float32)]
Step 359, Epoch 0 Metrics [array([0.55122304], dtype=float32), array([0.75], dtype=float32)]
Step 360, Epoch 0 Metrics [array([0.5273176], dtype=float32), array([0.859375], dtype=float32)]
Step 361, Epoch 0 Metrics [array([0.56935227], dtype=float32), array([0.75], dtype=float32)]
Step 362, Epoch 0 Metrics [array([0.5317521], dt

...

Step 379, Epoch 18 Metrics [array([0.15979673], dtype=float32), array([0.921875], dtype=float32)]
Step 380, Epoch 18 Metrics [array([0.11459792], dtype=float32), array([0.9375], dtype=float32)]
Step 381, Epoch 18 Metrics [array([0.16299556], dtype=float32), array([0.953125], dtype=float32)]
Step 382, Epoch 18 Metrics [array([0.13500303], dtype=float32), array([0.9375], dtype=float32)]
Step 383, Epoch 18 Metrics [array([0.07491885], dtype=float32), array([0.96875], dtype=float32)]
Step 384, Epoch 18 Metrics [array([0.1953106], dtype=float32), array([0.9375], dtype=float32)]
Step 385, Epoch 18 Metrics [array([0.21303813], dtype=float32), array([0.953125], dtype=float32)]
Step 386, Epoch 18 Metrics [array([0.07562278], dtype=float32), array([0.96875], dtype=float32)]
Step 387, Epoch 18 Metrics [array([0.17414883], dtype=float32), array([0.921875], dtype=float32)]
Step 388, Epoch 18 Metrics [array([0.11683853], dtype=float32), array([0.96875], dtype=float32)]
Step 389, Epoch 18 Metrics [ar

Step 75, Epoch 19 Metrics [array([0.06018678], dtype=float32), array([0.984375], dtype=float32)]
Step 76, Epoch 19 Metrics [array([0.08958469], dtype=float32), array([0.984375], dtype=float32)]
Step 77, Epoch 19 Metrics [array([0.07844989], dtype=float32), array([0.984375], dtype=float32)]
Step 78, Epoch 19 Metrics [array([0.18739657], dtype=float32), array([0.9375], dtype=float32)]
Step 79, Epoch 19 Metrics [array([0.08277286], dtype=float32), array([0.984375], dtype=float32)]
Step 80, Epoch 19 Metrics [array([0.08993088], dtype=float32), array([0.984375], dtype=float32)]
Step 81, Epoch 19 Metrics [array([0.17003617], dtype=float32), array([0.953125], dtype=float32)]
Step 82, Epoch 19 Metrics [array([0.0895129], dtype=float32), array([0.96875], dtype=float32)]
Step 83, Epoch 19 Metrics [array([0.06267974], dtype=float32), array([1.], dtype=float32)]
Step 84, Epoch 19 Metrics [array([0.16100642], dtype=float32), array([0.953125], dtype=float32)]
Step 85, Epoch 19 Metrics [array([0.1370

Step 163, Epoch 19 Metrics [array([0.15727228], dtype=float32), array([0.9375], dtype=float32)]
Step 164, Epoch 19 Metrics [array([0.15492767], dtype=float32), array([0.953125], dtype=float32)]
Step 165, Epoch 19 Metrics [array([0.1167566], dtype=float32), array([0.96875], dtype=float32)]
Step 166, Epoch 19 Metrics [array([0.09793407], dtype=float32), array([0.96875], dtype=float32)]
Step 167, Epoch 19 Metrics [array([0.03203741], dtype=float32), array([1.], dtype=float32)]
Step 168, Epoch 19 Metrics [array([0.12578], dtype=float32), array([0.921875], dtype=float32)]
Step 169, Epoch 19 Metrics [array([0.19041592], dtype=float32), array([0.9375], dtype=float32)]
Step 170, Epoch 19 Metrics [array([0.14859624], dtype=float32), array([0.9375], dtype=float32)]
Step 171, Epoch 19 Metrics [array([0.15635616], dtype=float32), array([0.953125], dtype=float32)]
Step 172, Epoch 19 Metrics [array([0.13458988], dtype=float32), array([0.9375], dtype=float32)]
Step 173, Epoch 19 Metrics [array([0.102

Step 251, Epoch 19 Metrics [array([0.24775594], dtype=float32), array([0.890625], dtype=float32)]
Step 252, Epoch 19 Metrics [array([0.1494328], dtype=float32), array([0.9375], dtype=float32)]
Step 253, Epoch 19 Metrics [array([0.08916634], dtype=float32), array([0.953125], dtype=float32)]
Step 254, Epoch 19 Metrics [array([0.1304022], dtype=float32), array([0.921875], dtype=float32)]
Step 255, Epoch 19 Metrics [array([0.09104057], dtype=float32), array([0.96875], dtype=float32)]
Step 256, Epoch 19 Metrics [array([0.09398338], dtype=float32), array([0.96875], dtype=float32)]
Step 257, Epoch 19 Metrics [array([0.09432297], dtype=float32), array([0.953125], dtype=float32)]
Step 258, Epoch 19 Metrics [array([0.07147381], dtype=float32), array([0.984375], dtype=float32)]
Step 259, Epoch 19 Metrics [array([0.12633793], dtype=float32), array([0.953125], dtype=float32)]
Step 260, Epoch 19 Metrics [array([0.19456682], dtype=float32), array([0.921875], dtype=float32)]
Step 261, Epoch 19 Metrics

Step 339, Epoch 19 Metrics [array([0.1434555], dtype=float32), array([0.953125], dtype=float32)]
Step 340, Epoch 19 Metrics [array([0.14391923], dtype=float32), array([0.953125], dtype=float32)]
Step 341, Epoch 19 Metrics [array([0.090916], dtype=float32), array([0.96875], dtype=float32)]
Step 342, Epoch 19 Metrics [array([0.15120314], dtype=float32), array([0.921875], dtype=float32)]
Step 343, Epoch 19 Metrics [array([0.11907858], dtype=float32), array([0.9375], dtype=float32)]
Step 344, Epoch 19 Metrics [array([0.13568264], dtype=float32), array([0.9375], dtype=float32)]
Step 345, Epoch 19 Metrics [array([0.09668042], dtype=float32), array([0.96875], dtype=float32)]
Step 346, Epoch 19 Metrics [array([0.2044672], dtype=float32), array([0.90625], dtype=float32)]
Step 347, Epoch 19 Metrics [array([0.20944536], dtype=float32), array([0.890625], dtype=float32)]
Step 348, Epoch 19 Metrics [array([0.19525596], dtype=float32), array([0.90625], dtype=float32)]
Step 349, Epoch 19 Metrics [arra

In [14]:
train_loop(fluid.default_main_program())

Step 0, Epoch 0 Metrics [array([0.7017218], dtype=float32), array([0.46875], dtype=float32)]
Step 1, Epoch 0 Metrics [array([0.6951672], dtype=float32), array([0.5], dtype=float32)]
Step 2, Epoch 0 Metrics [array([0.6955037], dtype=float32), array([0.5], dtype=float32)]
Step 3, Epoch 0 Metrics [array([0.6789076], dtype=float32), array([0.625], dtype=float32)]
Step 4, Epoch 0 Metrics [array([0.69754344], dtype=float32), array([0.46875], dtype=float32)]
Step 5, Epoch 0 Metrics [array([0.6881278], dtype=float32), array([0.546875], dtype=float32)]
Step 6, Epoch 0 Metrics [array([0.69201297], dtype=float32), array([0.515625], dtype=float32)]
Step 7, Epoch 0 Metrics [array([0.69392586], dtype=float32), array([0.5], dtype=float32)]
Step 8, Epoch 0 Metrics [array([0.6957596], dtype=float32), array([0.5], dtype=float32)]
Step 9, Epoch 0 Metrics [array([0.6977148], dtype=float32), array([0.453125], dtype=float32)]
Step 10, Epoch 0 Metrics [array([0.6925702], dtype=float32), array([0.515625], dty

Step 89, Epoch 0 Metrics [array([0.6817957], dtype=float32), array([0.53125], dtype=float32)]
Step 90, Epoch 0 Metrics [array([0.68029404], dtype=float32), array([0.578125], dtype=float32)]
Step 91, Epoch 0 Metrics [array([0.6714006], dtype=float32), array([0.65625], dtype=float32)]
Step 92, Epoch 0 Metrics [array([0.6700317], dtype=float32), array([0.671875], dtype=float32)]
Step 93, Epoch 0 Metrics [array([0.6881169], dtype=float32), array([0.546875], dtype=float32)]
Step 94, Epoch 0 Metrics [array([0.6787155], dtype=float32), array([0.71875], dtype=float32)]
Step 95, Epoch 0 Metrics [array([0.6802895], dtype=float32), array([0.640625], dtype=float32)]
Step 96, Epoch 0 Metrics [array([0.67221284], dtype=float32), array([0.703125], dtype=float32)]
Step 97, Epoch 0 Metrics [array([0.6781779], dtype=float32), array([0.546875], dtype=float32)]
Step 98, Epoch 0 Metrics [array([0.67966187], dtype=float32), array([0.640625], dtype=float32)]
Step 99, Epoch 0 Metrics [array([0.6776897], dtype

Step 176, Epoch 0 Metrics [array([0.65634155], dtype=float32), array([0.8125], dtype=float32)]
Step 177, Epoch 0 Metrics [array([0.65279853], dtype=float32), array([0.859375], dtype=float32)]
Step 178, Epoch 0 Metrics [array([0.643336], dtype=float32), array([0.828125], dtype=float32)]
Step 179, Epoch 0 Metrics [array([0.6574823], dtype=float32), array([0.8125], dtype=float32)]
Step 180, Epoch 0 Metrics [array([0.6460046], dtype=float32), array([0.828125], dtype=float32)]
Step 181, Epoch 0 Metrics [array([0.6592376], dtype=float32), array([0.734375], dtype=float32)]
Step 182, Epoch 0 Metrics [array([0.6405518], dtype=float32), array([0.90625], dtype=float32)]
Step 183, Epoch 0 Metrics [array([0.64380807], dtype=float32), array([0.84375], dtype=float32)]
Step 184, Epoch 0 Metrics [array([0.64386934], dtype=float32), array([0.734375], dtype=float32)]
Step 185, Epoch 0 Metrics [array([0.643873], dtype=float32), array([0.734375], dtype=float32)]
Step 186, Epoch 0 Metrics [array([0.6462786]

Step 263, Epoch 0 Metrics [array([0.60408115], dtype=float32), array([0.734375], dtype=float32)]
Step 264, Epoch 0 Metrics [array([0.6089387], dtype=float32), array([0.796875], dtype=float32)]
Step 265, Epoch 0 Metrics [array([0.6224022], dtype=float32), array([0.71875], dtype=float32)]
Step 266, Epoch 0 Metrics [array([0.6193839], dtype=float32), array([0.71875], dtype=float32)]
Step 267, Epoch 0 Metrics [array([0.6371695], dtype=float32), array([0.703125], dtype=float32)]
Step 268, Epoch 0 Metrics [array([0.62669694], dtype=float32), array([0.734375], dtype=float32)]
Step 269, Epoch 0 Metrics [array([0.60576594], dtype=float32), array([0.75], dtype=float32)]
Step 270, Epoch 0 Metrics [array([0.61050606], dtype=float32), array([0.8125], dtype=float32)]
Step 271, Epoch 0 Metrics [array([0.6220859], dtype=float32), array([0.6875], dtype=float32)]
Step 272, Epoch 0 Metrics [array([0.6292969], dtype=float32), array([0.6875], dtype=float32)]
Step 273, Epoch 0 Metrics [array([0.59882087], d

Step 352, Epoch 0 Metrics [array([0.5908958], dtype=float32), array([0.703125], dtype=float32)]
Step 353, Epoch 0 Metrics [array([0.54924387], dtype=float32), array([0.765625], dtype=float32)]
Step 354, Epoch 0 Metrics [array([0.55632794], dtype=float32), array([0.78125], dtype=float32)]
Step 355, Epoch 0 Metrics [array([0.556459], dtype=float32), array([0.796875], dtype=float32)]
Step 356, Epoch 0 Metrics [array([0.56488], dtype=float32), array([0.765625], dtype=float32)]
Step 357, Epoch 0 Metrics [array([0.5503895], dtype=float32), array([0.8125], dtype=float32)]
Step 358, Epoch 0 Metrics [array([0.5799819], dtype=float32), array([0.703125], dtype=float32)]
Step 359, Epoch 0 Metrics [array([0.55806696], dtype=float32), array([0.828125], dtype=float32)]
Step 360, Epoch 0 Metrics [array([0.5717249], dtype=float32), array([0.703125], dtype=float32)]
Step 361, Epoch 0 Metrics [array([0.6021418], dtype=float32), array([0.65625], dtype=float32)]
Step 362, Epoch 0 Metrics [array([0.54490983

...

Step 386, Epoch 18 Metrics [array([0.18398021], dtype=float32), array([0.90625], dtype=float32)]
Step 387, Epoch 18 Metrics [array([0.13014674], dtype=float32), array([0.953125], dtype=float32)]
Step 388, Epoch 18 Metrics [array([0.20777434], dtype=float32), array([0.875], dtype=float32)]
Step 389, Epoch 18 Metrics [array([0.16905528], dtype=float32), array([0.921875], dtype=float32)]
Step 390, Epoch 18 Metrics [array([0.09771139], dtype=float32), array([0.95], dtype=float32)]
Step 0, Epoch 19 Metrics [array([0.21565309], dtype=float32), array([0.90625], dtype=float32)]
Step 1, Epoch 19 Metrics [array([0.0883868], dtype=float32), array([0.96875], dtype=float32)]
Step 2, Epoch 19 Metrics [array([0.12650825], dtype=float32), array([0.953125], dtype=float32)]
Step 3, Epoch 19 Metrics [array([0.09196746], dtype=float32), array([0.953125], dtype=float32)]
Step 4, Epoch 19 Metrics [array([0.08319207], dtype=float32), array([0.984375], dtype=float32)]
Step 5, Epoch 19 Metrics [array([0.087483

Step 84, Epoch 19 Metrics [array([0.110245], dtype=float32), array([0.984375], dtype=float32)]
Step 85, Epoch 19 Metrics [array([0.17479381], dtype=float32), array([0.96875], dtype=float32)]
Step 86, Epoch 19 Metrics [array([0.1069304], dtype=float32), array([0.96875], dtype=float32)]
Step 87, Epoch 19 Metrics [array([0.11221132], dtype=float32), array([0.953125], dtype=float32)]
Step 88, Epoch 19 Metrics [array([0.13538128], dtype=float32), array([0.953125], dtype=float32)]
Step 89, Epoch 19 Metrics [array([0.10782921], dtype=float32), array([0.96875], dtype=float32)]
Step 90, Epoch 19 Metrics [array([0.1837343], dtype=float32), array([0.921875], dtype=float32)]
Step 91, Epoch 19 Metrics [array([0.12538081], dtype=float32), array([0.9375], dtype=float32)]
Step 92, Epoch 19 Metrics [array([0.07958788], dtype=float32), array([0.984375], dtype=float32)]
Step 93, Epoch 19 Metrics [array([0.12215166], dtype=float32), array([0.953125], dtype=float32)]
Step 94, Epoch 19 Metrics [array([0.085

Step 172, Epoch 19 Metrics [array([0.09310909], dtype=float32), array([0.984375], dtype=float32)]
Step 173, Epoch 19 Metrics [array([0.172143], dtype=float32), array([0.921875], dtype=float32)]
Step 174, Epoch 19 Metrics [array([0.10701519], dtype=float32), array([1.], dtype=float32)]
Step 175, Epoch 19 Metrics [array([0.06641509], dtype=float32), array([0.984375], dtype=float32)]
Step 176, Epoch 19 Metrics [array([0.09197991], dtype=float32), array([0.984375], dtype=float32)]
Step 177, Epoch 19 Metrics [array([0.1723504], dtype=float32), array([0.9375], dtype=float32)]
Step 178, Epoch 19 Metrics [array([0.14319664], dtype=float32), array([0.90625], dtype=float32)]
Step 179, Epoch 19 Metrics [array([0.0964779], dtype=float32), array([0.96875], dtype=float32)]
Step 180, Epoch 19 Metrics [array([0.14475547], dtype=float32), array([0.9375], dtype=float32)]
Step 181, Epoch 19 Metrics [array([0.15399066], dtype=float32), array([0.921875], dtype=float32)]
Step 182, Epoch 19 Metrics [array([0

Step 259, Epoch 19 Metrics [array([0.09249929], dtype=float32), array([0.984375], dtype=float32)]
Step 260, Epoch 19 Metrics [array([0.05923177], dtype=float32), array([1.], dtype=float32)]
Step 261, Epoch 19 Metrics [array([0.11905396], dtype=float32), array([0.984375], dtype=float32)]
Step 262, Epoch 19 Metrics [array([0.08352055], dtype=float32), array([1.], dtype=float32)]
Step 263, Epoch 19 Metrics [array([0.12171876], dtype=float32), array([0.96875], dtype=float32)]
Step 264, Epoch 19 Metrics [array([0.13750833], dtype=float32), array([0.96875], dtype=float32)]
Step 265, Epoch 19 Metrics [array([0.07522558], dtype=float32), array([0.984375], dtype=float32)]
Step 266, Epoch 19 Metrics [array([0.06902136], dtype=float32), array([1.], dtype=float32)]
Step 267, Epoch 19 Metrics [array([0.14092968], dtype=float32), array([0.921875], dtype=float32)]
Step 268, Epoch 19 Metrics [array([0.20371525], dtype=float32), array([0.921875], dtype=float32)]
Step 269, Epoch 19 Metrics [array([0.161

Step 346, Epoch 19 Metrics [array([0.27578402], dtype=float32), array([0.859375], dtype=float32)]
Step 347, Epoch 19 Metrics [array([0.10366221], dtype=float32), array([0.984375], dtype=float32)]
Step 348, Epoch 19 Metrics [array([0.13940072], dtype=float32), array([0.921875], dtype=float32)]
Step 349, Epoch 19 Metrics [array([0.1405544], dtype=float32), array([0.9375], dtype=float32)]
Step 350, Epoch 19 Metrics [array([0.26031333], dtype=float32), array([0.921875], dtype=float32)]
Step 351, Epoch 19 Metrics [array([0.09379388], dtype=float32), array([0.984375], dtype=float32)]
Step 352, Epoch 19 Metrics [array([0.08828169], dtype=float32), array([0.984375], dtype=float32)]
Step 353, Epoch 19 Metrics [array([0.15188473], dtype=float32), array([0.953125], dtype=float32)]
Step 354, Epoch 19 Metrics [array([0.14950043], dtype=float32), array([0.953125], dtype=float32)]
Step 355, Epoch 19 Metrics [array([0.12698439], dtype=float32), array([0.96875], dtype=float32)]
Step 356, Epoch 19 Metri